<a href="https://colab.research.google.com/github/yefry08/1791/blob/main/Class_1_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Shopping Assistant - Setup & Dependencies

**Engineering Logic**: We're building a sophisticated RAG (Retrieval-Augmented Generation) system that combines semantic search with LLM reasoning. The key dependencies are:

- **FAISS**: Facebook's similarity search library for efficient vector operations
- **SentenceTransformers**: For creating semantic embeddings that understand meaning, not just keywords  
- **LangChain**: Provides clean abstractions for LLM interactions and conversation management
- **Pydantic**: Ensures structured, validated responses from our LLM to prevent parsing errors

This foundation enables us to move beyond basic keyword matching to true semantic understanding of user queries.

In [ ]:
# AI Shopping Assistant with RAG and Prompt Engineering

# =============================================================================
# 1: SETUP AND DEPENDENCIES
# =============================================================================

# Install required packages
!pip install faiss-cpu langchain_community sentence-transformers --quiet

# Import libraries
import requests
import faiss
import numpy as np
import json
import re
from sentence_transformers import SentenceTransformer
from pydantic import BaseModel, ValidationError
from langchain.chat_models import ChatOpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


# Vector Search Engine Implementation

**Engineering Logic**: This is the core of our RAG system. We're implementing semantic search because traditional keyword search fails for complex queries like "gaming headphones" or "wireless audio devices."

**Key Design Decisions**:
- **Mixed Bread AI Embeddings**: Using `mxbai-embed-large-v1` for superior semantic understanding
- **Multi-field Indexing**: Combining title + description + category for richer context
- **FAISS IndexFlatL2**: L2 distance for similarity - works well for product recommendations
- **Dynamic Product Loading**: Fetching all 194 products (limit=0) instead of default 30

**Why This Matters**: A user searching "good headphones for gaming" should find gaming headsets even if the product description doesn't contain those exact words. Semantic search understands intent and context.

In [ ]:
# =============================================================================
# 2: RAG IMPLEMENTATION WITH FAISS
# =============================================================================

def setup_vector_search():
    """Set up semantic search using FAISS and sentence transformers."""

    # Load sentence transformer model
    model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

    # Fetch all products
    response = requests.get("https://dummyjson.com/products?limit=0")  # 0 = all products
    products = response.json()["products"]
    print(f"🔢 Loaded {len(products)} products into vector DB")

    # Create embeddings for TITLE + DESCRIPTION + CATEGORY
    text_to_embed = [f"{p['title']} {p['description']} {p['category']}" for p in products]
    embeddings = model.encode(text_to_embed)

    # Index embeddings in FAISS
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))

    return model, index, products

def semantic_search(query, model, index, products, k=3):
    """Perform semantic search for products."""
    # Use query prompt for better retrieval results
    query_embedding = model.encode([query], prompt_name="query")
    print(f"Query embedding shape: {query_embedding.shape}")

    # Ensure it's 2D array
    if len(query_embedding.shape) == 1:
        query_embedding = query_embedding.reshape(1, -1)

    D, I = index.search(query_embedding, k=k)

    results = [products[i] for i in I[0]]
    return results

# Test semantic search
model, index, products = setup_vector_search()
results = semantic_search("good headphones for gaming", model, index, products)

print("=== SEMANTIC SEARCH RESULTS ===")
for r in results:
    print(f"Found: {r['title']} - ${r['price']}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

🔢 Loaded 194 products into vector DB
Query embedding shape: (1, 1024)
=== SEMANTIC SEARCH RESULTS ===
Found: Beats Flex Wireless Earphones - $49.99
Found: Apple AirPods Max Silver - $549.99
Found: Gigabyte Aorus Men Tshirt - $24.99


# Product Data Access Layer

**Engineering Logic**: Clean separation of concerns - these functions handle all external API interactions with proper error handling and consistent interfaces.

**Design Patterns**:
- **Single Responsibility**: Each function has one clear purpose
- **Error Resilience**: All API calls wrapped with try/catch and graceful fallbacks
- **Flexible Parameters**: Functions accept both product names and IDs for maximum usability
- **Mock Checkout**: Simulates real e-commerce flow without actual payment processing

This layer abstracts away API complexity and provides a clean interface for our AI agent to use.

In [ ]:
# =============================================================================
# 3: API FUNCTIONS FOR PRODUCT DATA
# =============================================================================

def fetch_data(url):
    """Helper function to fetch data from API."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"❌ API Error: {e}")
        return None

def search_products(keyword):
    """Search for products by keyword."""
    return fetch_data(f"https://dummyjson.com/products/search?q={keyword}").get("products", [])

def get_products_by_category(category):
    """Get products from specific category."""
    return fetch_data(f"https://dummyjson.com/products/category/{category}").get("products", [])

def get_product_by_id(product_id):
    """Get specific product details."""
    return fetch_data(f"https://dummyjson.com/products/{product_id}")

def get_sorted_products(sort_by="price", limit=10):
    """Get sorted products."""
    return fetch_data(f"https://dummyjson.com/products?sort={sort_by}&limit={limit}").get("products", [])

def get_product_reviews(product_id):
    """Get product reviews."""
    return fetch_data(f"https://dummyjson.com/products/{product_id}/reviews").get("reviews", [])

def process_checkout(product_name=None, product_id=None):
    """Mock checkout process - handles both product name and ID."""
    if product_id:
        # Get product by ID to find the name
        product = get_product_by_id(product_id)
        if product:
            product_name = product['title']
        else:
            return f"❌ Product with ID {product_id} not found"

    if not product_name:
        return "❌ Please provide either product_name or product_id"

    return f"🛒 Proceeding to checkout for {product_name}. Click here: https://mystore.com/checkout/{product_name.replace(' ', '-').lower()}"


def semantic_search_tool(query, k=5, price_min=None, price_max=None, category=None):
    """Wrapper for semantic search to use as a tool."""
    return semantic_search(query, model, index, products, k=k)

# Update TOOLS dictionary
TOOLS = {
    "search_products": search_products,
    "semantic_search": semantic_search_tool,
    "get_products_by_category": get_products_by_category,
    "get_product_by_id": get_product_by_id,
    "get_sorted_products": get_sorted_products,
    "get_product_reviews": get_product_reviews,
    "process_checkout": process_checkout
}



# Response Validation & Structure

**Engineering Logic**: LLMs can be unpredictable in their output format. Pydantic models ensure our assistant always returns properly structured responses that our system can reliably parse.

**Why Pydantic**:
- **Type Safety**: Prevents runtime errors from malformed LLM responses
- **Automatic Validation**: Catches issues before they reach the user
- **JSON Parsing**: Handles edge cases like null values and malformed JSON
- **Clear Contracts**: Defines exactly what our AI should return

This is defensive programming - we assume the LLM might return unexpected formats and handle it gracefully.

In [ ]:
# =============================================================================
# 4: STRUCTURED RESPONSE MODEL
# =============================================================================

class ChatbotResponse(BaseModel):
    """Structured response format for the chatbot."""
    thinking: str
    response: str
    action: str | None = None
    parameters: dict | None = None

    @classmethod
    def from_json_string(cls, json_string: str):
        """Parse JSON string and handle null values.

        This function is needed, otherwise the LLM returns "null" as a string
        instead of actual None/null, which breaks Pydantic validation since
        it expects None but gets the literal string "null".
        """
        try:
            data = json.loads(json_string)
            if data.get("action") == "null":
                data["action"] = None
            return cls(**data)
        except json.JSONDecodeError:
            raise ValueError("Invalid JSON format from LLM.")

# The AI Agent

**Engineering Logic**: This is where everything comes together - the class that orchestrates semantic search, LLM reasoning, and conversation management.

**Architectural Highlights**:
- **Conversation Memory**: Tracks context across turns to avoid repeating failed searches
- **Tool Selection Intelligence**: The system prompt teaches the LLM when to use semantic vs. keyword search
- **Error Recovery**: If basic search fails, automatically falls back to semantic search
- **Natural Language Interface**: Users don't need to know about different search types

**The system prompt**:  explicitly teaches the LLM about its own limitations and guides it toward better tool selection.

In [ ]:
# =============================================================================
# 5: SMART SHOPPING ASSISTANT CLASS
# =============================================================================

class SmartShoppingAssistant:
    """Complete AI Shopping Assistant with RAG capabilities."""

    def __init__(self, openai_api_key, model="gpt-4o"):
        """Initialize the assistant."""
        self.llm = ChatOpenAI(model=model, openai_api_key=openai_api_key)
        self.conversation_history = []
        self.tools = TOOLS
        self.system_prompt = self._create_system_prompt()

        # Initialize RAG components
        self.model, self.index, self.products = setup_vector_search()

    def _create_system_prompt(self):
        """Create the system prompt for the AI assistant."""
        return """You are a smart shopping assistant that responds naturally to users. You have access to both basic keyword search AND advanced semantic search capabilities.

        **Available API Actions**:
        - "search_products" → Basic keyword search (exact text matches only)
        - "semantic_search" → Advanced AI semantic search (understands meaning, context, synonyms)
        - "get_products_by_category" → Fetch products from a specific category
        - "get_product_by_id" → Get specific product details
        - "get_sorted_products" → Get sorted products
        - "get_product_reviews" → Get customer reviews
        - "process_checkout" → Proceed to checkout

        - The basic "search_products" often returns EMPTY RESULTS for complex queries
        - You have a powerful semantic search that understands meaning - USE IT!
        - For queries like "gaming headphones", "wireless audio", "bluetooth earphones" - USE SEMANTIC SEARCH
        - Only use basic search for simple, exact product names

        **Semantic Search Parameters**:
        - query (required): what user is looking for (can be descriptive)
        - k (optional): number of results (default 5, max 10)
        - price_min, price_max (optional): price range filtering
        - category (optional): filter by specific category
        - rating_min (optional): minimum rating filter

        **Response Format**: Always respond in JSON:
        {
            "thinking": "your reasoning about the user's request and which search method to use - be honest about why you choose semantic vs basic search",
            "response": "your message to the user",
            "action": "function_name_or_null",
            "parameters": {}
        }

        **Search Strategy Guidelines**:
        - For descriptive/complex queries → USE "semantic_search" (e.g., "gaming headphones", "good sound quality", "wireless audio")
        - For exact product names → USE "search_products" (e.g., "iPhone 14", "MacBook Pro")
        - If basic search returns empty results → IMMEDIATELY try semantic_search as fallback
        - For greetings/general chat → action: null

        **VERY IMPORTANT**: If you keep using basic search for complex queries and getting empty results, you're failing the user. The semantic search is there for a reason - it understands context and meaning. Use it!
        """

    def _extract_json_from_response(self, llm_response):
        """Extract and clean JSON from LLM response."""
        if hasattr(llm_response, "content"):
            content = llm_response.content

            # Extract JSON from code blocks if present
            json_match = re.search(r"```json\n(.*?)\n```", content, re.DOTALL)
            if json_match:
                content = json_match.group(1)

            return content.strip()

        raise ValueError("Invalid LLM response format")

    def get_ai_decision(self, user_query):
        """Get AI decision on what action to take."""
        prompt = f"{self.system_prompt}\n\nConversation history: {json.dumps(self.conversation_history, indent=2)}\n\nUser: {user_query}"

        llm_response = self.llm.invoke(prompt)

        try:
            json_content = self._extract_json_from_response(llm_response)
            decision = ChatbotResponse.from_json_string(json_content)
            return decision
        except (ValidationError, ValueError) as e:
            print(f"❌ Invalid AI response: {e}")
            return None

    def execute_action(self, decision):
        """Execute the chosen API action."""
        if decision.action and decision.action in self.tools:
            print(f"🔧 Executing: {decision.action} with {decision.parameters}")
            result = self.tools[decision.action](**decision.parameters)
            print(f"🔍 Action result: {result}")  # Add this line for debugging
            return result
        return None

    def generate_final_response(self, action_result):
        """Generate natural response based on retrieved data."""
        if action_result:
            if isinstance(action_result, list) and len(action_result) == 0:
                return "I couldn't find any products matching your search. Could you try a different search term?"
            prompt = f"Based on this product data, respond naturally and show the products:\n{json.dumps(action_result, indent=2)}"
            response = self.llm.invoke(prompt)
            return response.content
        return "I couldn't retrieve any product information at the moment."

    def chat_turn(self, user_query):
        """Process one turn of conversation."""
        # Get AI decision
        decision = self.get_ai_decision(user_query)
        if not decision:
            return "Sorry, I had trouble understanding that."

        # Execute action if needed
        action_result = self.execute_action(decision)

        # Generate final response
        if action_result:
            final_response = self.generate_final_response(action_result)
        else:
            final_response = decision.response

        # Update conversation history
        self.conversation_history.append({
            "user": user_query,
            "ai": final_response,
            "action": decision.action,
            "result": action_result
        })

        return final_response

    def start_interactive_chat(self):
        """Start interactive chat session."""
        print("🛒 Welcome to the AI Store! How can I help you today?")
        print("(Type 'exit', 'quit', or 'bye' to end)")

        while True:
            user_input = input("\nYou: ").strip()

            if user_input.lower() in ["exit", "quit", "bye"]:
                print("👋 Thanks for visiting! Have a great day.")
                break

            if not user_input:
                continue

            response = self.chat_turn(user_input)
            print(f"\n🤖 AI: {response}")

# Interactive Demo & Validation

**Engineering Logic**: Practical demonstration of the complete system with both programmatic testing and interactive chat interface.

**Testing Strategy**:
- **Single Turn Testing**: Validate core functionality with isolated queries
- **Interactive Mode**: Real-world user experience testing
- **Conversation Flow**: Ensures context preservation across multiple turns

This section proves the system works end-to-end and provides a clear usage pattern for integration into larger applications.

In [ ]:
# =============================================================================
# 6: USAGE EXAMPLE
# =============================================================================

# Initialize the assistant (replace with your actual API key)
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Create assistant instance
assistant = SmartShoppingAssistant(openai_api_key=OPENAI_API_KEY)

# Example: Single chat turn (for testing)
print("=== TESTING SINGLE CHAT TURN ===")
response = assistant.chat_turn("I'm looking for gaming headphones")
print(f"AI Response: {response}")

# Uncomment to start interactive chat
assistant.start_interactive_chat()

🔢 Loaded 194 products into vector DB
=== TESTING SINGLE CHAT TURN ===
🔧 Executing: semantic_search with {'query': 'gaming headphones'}
Query embedding shape: (1, 1024)
🔍 Action result: [{'id': 107, 'title': 'Beats Flex Wireless Earphones', 'description': 'The Beats Flex Wireless Earphones offer a comfortable and versatile audio experience. With magnetic earbuds and up to 12 hours of battery life, they are ideal for everyday use.', 'category': 'mobile-accessories', 'price': 49.99, 'discountPercentage': 5.73, 'rating': 4.24, 'stock': 50, 'tags': ['electronics', 'wireless earphones'], 'brand': 'Beats', 'sku': 'MOB-BEA-BEA-107', 'weight': 8, 'dimensions': {'width': 17.86, 'height': 25.74, 'depth': 23.09}, 'warrantyInformation': '1 year warranty', 'shippingInformation': 'Ships in 1-2 business days', 'availabilityStatus': 'In Stock', 'reviews': [{'rating': 2, 'comment': 'Disappointing product!', 'date': '2025-04-30T09:41:02.053Z', 'reviewerName': 'William Gonzalez', 'reviewerEmail': 'william

# Task: Add LLM-Based Reranker to Semantic Search



1. Inside SmartShoppingAssistant, implement:

    def llm_rerank_results(self, query: str, candidates: list[dict]) -> list[dict]:

    - Prompt the LLM to decide which candidates are relevant.
    - You must **define and explain the output format** in your prompt.
    - Parse the LLM response safely (e.g., strip ``` if present, use json.loads).
    - Return only the list of filtered results (in your chosen format).



2. Modify semantic_search_tool to:

    - Call semantic_search(...) first.
    - Then rerank using assistant.llm_rerank_results(query, candidates).
    - Return only the filtered list.



3. Test your implementation with:

    response = assistant.chat_turn("I'm looking for a powerful laptop for video editing")

    print(response)